# Projeto Técnicas de Programação
## Grupo 7 - Turma 1007 - ADA Santader Coders 2023 Data Science
---
## Membros
### Clara Ferreira Batista
### Laura Muglia
### Luana Ferraz
### Pedro Elias
### Rafael Couto de Oliveira
---

Nosso repositório :
https://github.com/claraferreirabatista/analise_dos_microdados_do_enem.git

---
## *Introdução*

* Motivo do trabalho: Analise dos Microdados do Enem;
* Contexto da analise do trabalho: Analise do impacto da COVID-19 nos números da região Centro-oeste referente aos anos de 2018 a 2022;
* Onde encontrar os dados : https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/enem.

### Divisão de tarefas

* Escolha do tópico: Laura Muglia
* Extração dos dados e amostras: Pedro Elias
* Carregamento, limpeza e tratativa dos dados: Rafael Couto
* Elaboração das perguntas: Luana Ferraz e Laura Muglia
* Análise exploratória: Pedro Elias, Rafael Couto
* Elaboração de gráficos: Pedro Elias, Luana Ferraz
* BugFix, reprodutibilidade e repositório: Clara Ferreira Batista
* Conclusões:


---
## Importando bibliotecas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Carregamento dos dados

In [ ]:
# Importe os arquivos Parquet para DataFrames Pandas
dados_2018_co = pd.read_parquet('dados_co/ENEM_2018_CENTROOESTE.parquet')
dados_2019_co = pd.read_parquet('dados_co/ENEM_2019_CENTROOESTE.parquet')
dados_2020_co = pd.read_parquet('dados_co/ENEM_2020_CENTROOESTE.parquet')
dados_2021_co = pd.read_parquet('dados_co/ENEM_2021_CENTROOESTE.parquet')
dados_2022_co = pd.read_parquet('dados_co/ENEM_2022_CENTROOESTE.parquet')

In [ ]:
# Concatenar os DataFrames
dados = pd.concat([dados_2018_co, dados_2019_co, dados_2020_co, dados_2021_co, dados_2022_co], ignore_index=True)
dados = dados.set_index('NU_INSCRICAO')
#Configurar pandas para exibir todas as colunas
pd.set_option('display.max_columns', None)

O dataset original foi carregado com 2.018.780 linhas e 76 colunas.

 ---
# 1. Limpeza dos dados
Exclusão de linhas e colunas não interessantes, unificação e redução de colunas que representam a mesma informação, tratativa de váriaveis e export do `dataset` limpo e tratado.

## 1.1 Exclusão de Colunas
Exclusão de colunas que não se mostraram importantes para análise.

In [ ]:
#Exclusão das colunas de questões socioeconomicas não interessantes, respostas e gabaritos

#Colunas de questões
dados = dados.iloc[:,:57]

#Colunas de gabarito
col_gabaritos = ['TX_GABARITO_MT','TX_GABARITO_LC','TX_GABARITO_CH','TX_GABARITO_CN']
dados.drop(columns=col_gabaritos, inplace=True)

#Colunas de respostas
col_respostas = ['TX_RESPOSTAS_MT','TX_RESPOSTAS_LC', 'TX_RESPOSTAS_CH','TX_RESPOSTAS_CN']
dados.drop(columns=col_respostas, inplace=True)

In [ ]:
#Exclusão de colunas não importantes para a análise

#Estado Civil
dados.drop(columns='TP_ESTADO_CIVIL', inplace=True)

#Tipo de instituição
dados.drop(columns='TP_ENSINO', inplace=True)

#Situação da escola
dados.drop(columns='TP_SIT_FUNC_ESC', inplace=True)

#codigos do municipio/estado
col_codigoslocais = ['CO_MUNICIPIO_ESC','CO_UF_ESC','CO_MUNICIPIO_PROVA','CO_UF_PROVA']
dados.drop(columns=col_codigoslocais, inplace=True)

#codigo da cor da prova
col_cores = ['CO_PROVA_CN','CO_PROVA_CH','CO_PROVA_LC','CO_PROVA_MT']
dados.drop(columns=col_cores, inplace=True)

#competencias redação
col_competencias = ['NU_NOTA_COMP1','NU_NOTA_COMP2','NU_NOTA_COMP3','NU_NOTA_COMP4','NU_NOTA_COMP5']
dados.drop(columns=col_competencias, inplace=True)


In [ ]:
#Exclusão das linhas de treineiros e da coluna
dados = dados[dados['IN_TREINEIRO'] == 0]
dados.drop(columns='IN_TREINEIRO', inplace=True)

## 1.2 Unificação de Colunas
Unificação de colunas multiplas em colunas singulares.

In [ ]:
#Unir presenças
#0	Faltou à prova
#1	Presente na prova
#2	Eliminado na prova

dados['TP_PRESENCA_MT'] =  dados['TP_PRESENCA_MT'].astype(str)
dados['TP_PRESENCA_CN'] =  dados['TP_PRESENCA_CN'].astype(str)
dados['TP_PRESENCA_CH'] =  dados['TP_PRESENCA_CH'].astype(str)
dados['TP_PRESENCA_LC'] =  dados['TP_PRESENCA_LC'].astype(str)


#Soma os TP_PRESENCA_CN, TP_PRESENCA_CH, TP_PRESENCA_LC,TP_PRESENCA_MT
dados['TP_PRESENCA'] = dados['TP_PRESENCA_CN'] + dados['TP_PRESENCA_CH'] + dados['TP_PRESENCA_LC'] + dados['TP_PRESENCA_MT']

#deletar as colunas de presença
col_presenca = ['TP_PRESENCA_MT','TP_PRESENCA_CN','TP_PRESENCA_CH','TP_PRESENCA_LC']
dados.drop(columns=col_presenca, inplace=True)

In [ ]:
#Analisando dados unidos
prop_presenca = dados['TP_PRESENCA'].value_counts(normalize=True)*100

#printando
graf1 = prop_presenca.plot(kind='bar')
plt.title('Proporções dos Valores Únicos na Coluna TP_PRESENCA')
plt.xlabel('Valor')
plt.ylabel('Proporção (%)')
#plt.xticks(rotation=0)
graf1.bar_label(graf1.containers[0], fmt='%.2f%%')
plt.tight_layout()
plt.show()

Os valores da coluna `TP_PRESENCA` serão transformados em `bool` e serão tratados da seguinte forma:
* `1 1 1 1` : Aluno Presente, 4 presenças (`TRUE`)
* `demais valores`: Aluno Ausente ou eliminado em uma ou mais provas (`FALSE`)

In [ ]:
dados['TP_PRESENCA'] = dados['TP_PRESENCA'].map({'1.01.01.01.0': True})
dados['TP_PRESENCA'] = dados['TP_PRESENCA'].fillna(False)

In [ ]:
#Nota Geral - Criar nova coluna somando as notas
dados['NU_NOTA_GERAL']=dados['NU_NOTA_CN'] + dados['NU_NOTA_CH'] + dados['NU_NOTA_LC'] + dados['NU_NOTA_MT']

## 1.3 Tratativa dos dados

Elaboração das tabelas "de-para" para valores que estão como "flag" ou valores numéricos, porém representam variaveis categóricas,substituição de flags por valores em colunas com baixa variabilidade e correção de formatos.

#### 1.3.1 Tabelas "De-para"

Elaboração das tabelas "de-para" para valores que estão como "flag" ou valores numéricos, porém representam variaveis categóricas. Usado para variáveis de alta variabilidade.

In [ ]:
#Tabela de/para: Faixa etária

# Dados para a coluna #TP_FAIXA_ETARIA
dp_idades = {
    'TP_FAIXA_ETARIA': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                         11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
    'Faixa Etária': ['Menor de 17 anos', '17 anos', '18 anos', '19 anos', '20 anos',
                     '21 anos', '22 anos', '23 anos', '24 anos', '25 anos',
                     'Entre 26 e 30 anos', 'Entre 31 e 35 anos', 'Entre 36 e 40 anos', 'Entre 41 e 45 anos',
                     'Entre 46 e 50 anos', 'Entre 51 e 55 anos', 'Entre 56 e 60 anos',
                     'Entre 61 e 65 anos', 'Entre 66 e 70 anos', 'Maior de 70 anos'],
    'Média idade': [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 28, 33, 38, 43, 48, 53, 58, 63, 68, 72]
}

dp_idades = pd.DataFrame(dp_idades).set_index('TP_FAIXA_ETARIA')

In [ ]:
#Tabela de/para: Cor/Raça

#Dados para a coluna #TP_COR_RACA
dp_raca = {
    'TP_COR_RACA': [0, 1, 2, 3, 4, 5, 6],
    'Cor/Raça': ['Não declarado', 'Branca', 'Preta', 'Parda', 'Amarela', 'Indígena', 'Não dispõe da informação']
}
dp_raca = pd.DataFrame(dp_raca).set_index('TP_COR_RACA')

In [ ]:
#Tabela de/para: Nacionalidade

#Dados para a coluna TP_NACIONALIDADE
dp_nacionalidade = {
    'TP_NACIONALIDADE': [0, 1, 2, 3, 4],
    'Nacionalidade_info': ['Não informado', 'Brasileiro(a)', 'Brasileiro(a) Naturalizado(a)',
                      'Estrangeiro(a)', 'Brasileiro(a) Nato(a), nascido(a) no exterior']
}

dp_nacionalidade = pd.DataFrame(dp_nacionalidade).set_index('TP_NACIONALIDADE')

In [ ]:
#Tabela de/para: Situação conclusão

#Dados para a coluna TP_ST_CONCLUSAO
dp_conclusao = {
    'TP_ST_CONCLUSAO': [1, 2, 3, 4],
    'Situação do Ensino Médio': ['Já concluí o Ensino Médio',
                                 'Estou cursando e concluirei o Ensino Médio no Ano da Prova',
                                 'Estou cursando e concluirei o Ensino Médio após o Ano da Prova',
                                 'Não concluí e não estou cursando o Ensino Médio']
}

dp_conclusao = pd.DataFrame(dp_conclusao).set_index('TP_ST_CONCLUSAO')

In [ ]:
#Tabela de/para: Ano Conclusão

# Dados para a coluna #TP_ANO_CONCLUIU
dp_ano_conclusao = {
    'TP_ANO_CONCLUIU': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                         11, 12, 13, 14, 15, 16],
    'Ano Concluiu': ['Não informado', '2021', '2020', '2019', '2018', '2017', '2016',
                     '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008',
                     '2007', 'Antes de 2007']
}

dp_ano_conclusao = pd.DataFrame(dp_ano_conclusao).set_index('TP_ANO_CONCLUIU')

In [ ]:
#Tabela de/para: Status Redação

# Dados para a coluna TP_STATUS_REDACAO

dp_redacao = {
    'TP_STATUS_REDACAO': [1, 2, 3, 4, 6, 7, 8, 9],
    'Status Redacao': ['Sem problemas', 'Anulada', 'Cópia Texto Motivador', 'Em Branco',
                       'Fuga ao tema', 'Não atendimento ao tipo textual', 'Texto insuficiente', 'Parte desconectada']
}

dp_redacao = pd.DataFrame(dp_redacao).set_index('TP_STATUS_REDACAO')

#### 1.3.2 Substituição de variáveis
Substituição de flags por valores em colunas com baixa variabilidade.

In [ ]:
#Substituição dos valores na coluna TP_ESCOLA

dados['TP_ESCOLA'] = dados['TP_ESCOLA'].map({1:'Não Respondeu', 2:'Pública', 3:'Privada'})

In [ ]:
#Substituição dos valores na coluna TP_LINGUA

dados['TP_LINGUA'] = dados['TP_LINGUA'].map({0:'Inglês', 1:'Espanhol'})

In [ ]:
#Substituição dos valores na coluna TP_DEPENDENCIA_ADM_ESC

dados['TP_DEPENDENCIA_ADM_ESC'] = dados['TP_DEPENDENCIA_ADM_ESC'].map({1:'Federal', 2:'Estadual', 3:'Municipal',4:'Privada'})


In [ ]:
#Substituição dos valores na coluna TP_LOCALIZACAO_ESC

dados['TP_LOCALIZACAO_ESC'] = dados['TP_LOCALIZACAO_ESC'].map({1:'Urbana', 2:'Rural'})

#### 1.3.3 Tratar dados nulos
Remoção de dados ou alocação de valores para dados nulos.

In [ ]:
#excluir linhas com presença nula
dados.dropna(subset=['TP_PRESENCA'],inplace=True)

In [ ]:
#excluir linhas com questionarios socio economicos nulos q001, q002 etc
dados.dropna(subset=['Q001'],inplace=True)
dados.dropna(subset=['Q002'],inplace=True)
dados.dropna(subset=['Q003'],inplace=True)
dados.dropna(subset=['Q004'],inplace=True)
dados.dropna(subset=['Q005'],inplace=True)
dados.dropna(subset=['Q006'],inplace=True)

In [ ]:
#Atribuir 0 para notas nulas, NU_NOTA_CN, NU_NOTA_CH, NU_NOTA_LC,NU_NOTA_MT, NU_NOTA_REDACAO, NU_NOTA_GERAL

# Lista das colunas de notas
col_notas = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO', 'NU_NOTA_GERAL']

# Preencher 0 para os valores nulos dessas colunas
dados[col_notas] = dados[col_notas].fillna(0)

In [ ]:
#atribuir 2 para redações nulas  - anulada TP_STATUS_REDACAO

dados['TP_STATUS_REDACAO'] = dados['TP_STATUS_REDACAO'].fillna(2)

#### 1.3.4 Ajustar Formatos
Corrigir tipos das colunas do dataset


In [ ]:
#converter TP_PRESENCA para bool
dados['TP_PRESENCA']=dados['TP_PRESENCA'].astype(bool)

## 1.4 Exportar Base tratada

In [ ]:
#exportar um csv unico com todos os dados tratados
dados.to_parquet('ENEM_CO_TRATADO.parquet', index=False)

---
# 2. Análise Exploratória das variáveis
Visualizações de algumas variáveis para possíveis insights.


In [ ]:
# Importar bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
#Carregando direto os Dados Tratados:
dados_tratados = pd.read_parquet('ENEM_CO_TRATADO.parquet')
dados_tratados = dados_tratados.drop(columns=['Q007'])


## 2.1 Variaveis Qualitativas
Análise exploratória de algumas variáveis qualitativas.

### Inscritos
Visualização da relação de inscritos e presentes ao longo dos anos.

In [ ]:
#Plotar QUANTIDADE_INSCRITOS X ANO NU_ANO
inscritos = pd.DataFrame(dados_tratados['NU_ANO'].value_counts())
presente = pd.DataFrame(dados_tratados[dados_tratados['TP_PRESENCA'] == True]['NU_ANO'].value_counts())
alunos = pd.concat([inscritos.rename(columns = {'NU_ANO':'Inscritos'}),presente.rename(columns = {'NU_ANO':'Presentes'})],axis = 1)
alunos.sort_index().plot(kind='bar')
#alunos.sort_index()


### Faixa Etária


In [ ]:
#Plotar quantidades x faixas etarias dos Inscritos

Faixa_Etaria = pd.DataFrame(dados_tratados['TP_FAIXA_ETARIA'].value_counts())
faixa = pd.concat([Faixa_Etaria.rename(columns = {'count':'Inscritos'}),dp_idades], axis = 1 )
faixa.sort_values(by='Inscritos' ).plot.barh(x = 'Faixa Etária', y = 'Inscritos')

In [ ]:
#Plotar quantidades x faixas etarias por ano

Faixa_Etaria_Ano = dados_tratados.merge(dp_idades.reindex(), on = 'TP_FAIXA_ETARIA', how = 'left')
Faixa_Etaria_Ano = Faixa_Etaria_Ano[['Faixa Etária', 'NU_ANO']].value_counts()
Faixa_Etaria_Ano = Faixa_Etaria_Ano.unstack()
cmap = sns.cm.rocket_r
sns.heatmap(Faixa_Etaria_Ano, cmap = cmap)
#Faixa_Etaria_Ano.sort_index().plot.barh( stacked = True)


### Tipo de Escola

In [ ]:
#plotar tipo de escola x anos (barras duplas)
tipo_escola = dados_tratados[['NU_ANO', 'TP_ESCOLA']].value_counts()
tipo_escola = tipo_escola.unstack()
tipo_escola.plot.barh()


### Língua

In [ ]:
#plotar quantidades linguas x anos (barras duplas)
tipo_lingua = dados_tratados[['NU_ANO', 'TP_LINGUA']].value_counts()
tipo_lingua = tipo_lingua.unstack()
tipo_lingua.plot.barh()

### Status Redação

In [ ]:
#plotar status redação geral (sem ser por ano, apenas tipos)
tipo_redacao = dados_tratados.merge(dp_redacao.reindex(), on = 'TP_STATUS_REDACAO', how = 'left')
tipo_redacao = tipo_redacao[['NU_ANO' , 'Status Redacao' ]].value_counts(normalize=True) * 100
tipo_redacao = tipo_redacao.unstack()
tipo_redacao.plot.bar()

### Presença

##Análise da Amostra - Estatística

###Sample Size

In [ ]:
#Checar se a amostragem é grande o suficiente:
# População do Centro-Oeste: 14058094 habitantes
N = 679_108 #populacao alunos do médio e médio tecnico do Centro-Oeste - abres.org.br/estatisticas/
p = 0.5 # proporção da população
z_95 = 1.96 # para um nível de confiança de 95%
z_99 = 2.58 # para um nível de confiança de 99%
e = 0.05 #margem de erro esperada
amostra = ((z_99**2 * p*(1-p))/(e**2))/(1+(z_99**2 * p*(1-p))/(e**2 * N))
amostra

## Variáveis Quantitativas

### Notas Gerais

In [ ]:
#Distribuicao normal?
#nota geral nao inclui a redacao
#Shapiro Test:
import numpy as np
from scipy.stats import shapiro
#import warnings
#warnings.filterwarnings('ignore')

filtro_notageral = dados_tratados['NU_NOTA_GERAL'] > 0
anos = [2018,2019,2020,2021,2022]
fig, axes = plt.subplots(nrows=2, ncols=3,figsize=(10, 10))
i = 0
j = 0
for ano in anos:
  filtro2 = dados_tratados['NU_ANO'] == ano
  amostra = pd.DataFrame(dados_tratados.loc[filtro_notageral & filtro2]['NU_NOTA_GERAL'])/4
  amostra.rename(columns={'NU_NOTA_GERAL':ano}).plot(kind = 'hist', ax=axes[i,j])
  if j <= 1 :
    j = j+1
  else:
    j = 0
    i = i+1

  pvalue = shapiro(amostra)[1]
  if pvalue > 0.05:
    print(f'Amostra de {ano} é distribuida normalmente com pvalue de {pvalue}')
  elif pvalue < 0.05:
    print(f'Amostra de {ano} NÃO é distribuida normalmente com pvalue de {pvalue}')





In [ ]:
notas = dados_tratados.loc[filtro_notageral][['NU_NOTA_GERAL','NU_ANO']]

notas['NU_NOTA_GERAL'] = notas['NU_NOTA_GERAL']/4

media = notas.groupby('NU_ANO').agg([np.mean, np.std])

media2 = media['NU_NOTA_GERAL']
media2.rename(columns={'mean':'Média Notas'}).plot(kind = 'barh', xerr = 'std')


In [ ]:
# plotar grafico de linha da média das notas ao longo do tempo
notas = dados_tratados.loc[filtro_notageral][['NU_NOTA_GERAL','NU_ANO']]
fig = go.Figure()
fig.add_trace(go.Box(
    x = notas['NU_ANO'],
    y = notas['NU_NOTA_GERAL']/4
))

#sns.boxplot(data = notas, x = 'NU_ANO', y = 'NU_NOTA_GERAL')
#media = pd.DataFrame(dados_tratados[['NU_ANO','NU_NOTA_GERAL']])
#media

### Notas Redação

In [ ]:
notas_redacao = dados_tratados.loc[filtro_notageral][['NU_NOTA_GERAL','NU_ANO']]

media_redacao = notas.groupby('NU_ANO').agg([np.mean, np.std])

media_redacao2 = media_redacao['NU_NOTA_GERAL']
media_redacao2.rename(columns={'mean':'Média Notas'}).plot(kind = 'barh', xerr = 'std')

In [ ]:
# plotar grafico de linha da média das notas ao longo do tempo
filtro_redacao = dados_tratados['NU_NOTA_REDACAO'] > 0
notas_redacao = dados_tratados.loc[filtro_redacao][['NU_NOTA_REDACAO','NU_ANO']]
fig = go.Figure()
fig.add_trace(go.Box(
    x = notas_redacao['NU_ANO'],
    y = notas_redacao['NU_NOTA_REDACAO']
))


### Correlação entre notas gerais e Notas Redação

In [ ]:
#plotar scatterplot notas gerais x notas redaçaão

notas_correlacao = dados_tratados.loc[filtro_notageral & filtro_redacao][['NU_NOTA_GERAL','NU_NOTA_REDACAO']]
notas_correlacao['NU_NOTA_GERAL'] = notas_correlacao['NU_NOTA_GERAL']/4

notas_correlacao.corr()


In [ ]:
#plotar scatterplot notas LC (linguagens e codigos) x notas redaçaão (hipotese, quem vai bem em portugues vai bem em redação)
notas_correlacao_LC = dados_tratados.loc[filtro_notageral & filtro_redacao][['NU_NOTA_LC','NU_NOTA_REDACAO']]
notas_correlacao_LC.corr()


---
# 3. PERGUNTAS



##1 Como a quantidade de faltas mudou ao longo do tempo?

In [ ]:
ausentes = dados_tratados[dados_tratados['TP_PRESENCA'] == False][['NU_ANO','SG_UF_PROVA']].value_counts()
ausentes.unstack().plot.barh(stacked = True)




##2

In [ ]:
#Código
#ver a parte dos tipos de escola la de cima


##3

In [ ]:
#Código
inscritos_sit_EM = dados_tratados.merge(dp_conclusao.reindex(), on = 'TP_ST_CONCLUSAO', how = 'left')
inscritos_sit_EM = inscritos_sit_EM[['NU_ANO','Situação do Ensino Médio']].value_counts()
inscritos_sit_EM.unstack().plot.bar(stacked=True, figsize = (15,8)).legend(loc=0)

#presente = pd.DataFrame(dados_tratados[dados_tratados['TP_PRESENCA'] == True]['NU_ANO'].value_counts())
#alunos = pd.concat([inscritos.rename(columns = {'NU_ANO':'Inscritos'}),presente.rename(columns = {'NU_ANO':'Presentes'})],axis = 1)
#alunos.sort_index().plot(kind='bar')

In [ ]:
#Entre os Terceiro Anistas:
#Código
filtro_terceiroanista = dados_tratados['TP_ST_CONCLUSAO'] == 2
terc_sit_EM = dados_tratados.loc[filtro_terceiroanista & filtro_redacao][['NU_NOTA_REDACAO','NU_ANO']]
media_terceiroanista = terc_sit_EM.groupby('NU_ANO').agg([np.mean, np.std])
media_terceiroanista2 = media_terceiroanista['NU_NOTA_REDACAO']
#renomeando colunas
media_terceiroanista2 = media_terceiroanista2.rename(columns={'mean':'mean 3a','std':'std 3a'})
media_redacao_compara = media_redacao2.rename(columns={'mean':'mean geral','std':'std geral'})

grafico_redacao = pd.concat([media_terceiroanista2,media_redacao_compara], axis = 1).reset_index()
grafico_redacao.plot(x = 'NU_ANO',y=["mean 3a", "mean geral"],  style='.-' , figsize = (15,8), xticks =grafico_redacao['NU_ANO'])
y1 = np.array(grafico_redacao['mean 3a']) - np.array(grafico_redacao['std 3a'])
y2 = np.array(grafico_redacao['mean 3a']) + np.array(grafico_redacao['std 3a'])
plt.fill_between(grafico_redacao['NU_ANO'],y1,y2,alpha=.1)
y1 = np.array(grafico_redacao['mean geral']) - np.array(grafico_redacao['std geral'])
y2 = np.array(grafico_redacao['mean geral']) + np.array(grafico_redacao['std geral'])
plt.fill_between(grafico_redacao['NU_ANO'],y1,y2,alpha=.1)




## 4. Estados que fazem divisa com países que falam espanhol tem maior escolha pela linha espanhola?




In [ ]:
#Código

tipo_lingua = dados_tratados[['NU_ANO', 'TP_LINGUA']].value_counts()
tipo_lingua = tipo_lingua.unstack()
tipo_lingua.plot.barh()

---
# 4. Conclusão

## Dificuldades e aprendizados


*   Pouco tempo para decisão do dataset
*   Dificuldade em trabalhar com dataset muito grande (garantir a reprodutibilidade)
* Dificuldade em trabalhar num ambiente colaborativo e versionado.



## Pontos Positivos


*   Equipe multidisciplinar
*   Troca de experiências



## Melhorias futuras
* Realizar testes estatisticos de normalidade
* Realizar testes estatisticos de hipótese